<a href="https://colab.research.google.com/github/lweislo/UCI_data/blob/main/uci_rider_ranking_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This script requires selenium to get past the UCI frames
from time import sleep
#This script requires selenium to get past the UCI frames
import requests
import re
import urllib3
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import files
from time import sleep
# !apt-get update
# !apt-get install chromium chromium-chromedriver
!pip3 install selenium
# !sudo apt -y update
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from io import StringIO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 14.9 MB/s eta 0:00:00


In [ ]:
!apt list --upgradable

In [ ]:
def init_browser():
    # Mac-specific browser init
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=options)

    return driver


# **Analysis of three-year UCI World Rankings**

In [ ]:
base_url = 'https://dataride.uci.org'

In [ ]:
#Get the UCI year and week IDs to choose ranking year (if desired)
url='https://dataride.uci.org/iframe/GetDisciplineSeasons/?disciplineId=10'
data = requests.get(url).json()


In [ ]:
#create a dictionary of code-year
year_id = [i['Id'] for i in data]
year = [i['Name'] for i in data]
year_dict = dict(zip(year_id, year))
year_dict

{444: '2025',
 432: '2024',
 414: '2023',
 159: '2022',
 150: '2021',
 145: '2020',
 132: '2019',
 89: '2018',
 19: '2017',
 15: '2016',
 7: '2015',
 85: '2014',
 84: '2013',
 83: '2012',
 82: '2011',
 81: '2010',
 80: '2009'}

In [ ]:
#Get the IDs for each week in each desired year
week_id = []
week_name = []
week_year = []
for y in year_id[0:1]:
    url = f'https://dataride.uci.org/iframe/GetRankingMoments/?disciplineId=10&disciplineSeasonId=444&rankingId=1'
    data2 = requests.get(url).json()
    for i in data2:
        if i['Id']!=0:
            week_id.append(i['Id'])
            week_name.append(i['Name'])
            week_year.append(y)


In [ ]:
#Choose the most recent ID for the ranking URL - it doesn't always work because the UCI
week_dict = dict(zip(week_id, week_name))
week_dict

{189398: '18/03/2025',
 189241: '11/03/2025',
 189039: '04/03/2025',
 188995: '25/02/2025',
 188731: '18/02/2025',
 188546: '11/02/2025',
 188398: '04/02/2025',
 188186: '28/01/2025',
 187964: '21/01/2025',
 187678: '14/01/2025',
 187563: '07/01/2025',
 187301: '31/12/2024',
 187206: '24/12/2024',
 186969: '17/12/2024',
 186732: '10/12/2024',
 186499: '03/12/2024',
 186266: '26/11/2024',
 186034: '19/11/2024',
 185802: '12/11/2024',
 185627: '05/11/2024',
 185306: '29/10/2024',
 185034: '22/10/2024'}

In [ ]:
#Get the URLs for each week's rankings @@ IT IS ANNOYING because the URL doesn't expose the data for each, it only exposes most recent.
# url_list = []
# for index, value in enumerate(week_id):
#     w = week_id[index]
#     y = week_year[index]
#     url = f'https://dataride.uci.org/iframe/RankingDetails/238?disciplineId=10&groupId=1&momentId={w}&disciplineSeasonId={y}&rankingTypeId=2&categoryId=22&raceTypeId=0'
#     url_list.append(url)

# url_list[:10]



In [ ]:
# URLs for latest rankings of each year
url_dict = {'2023':'https://dataride.uci.org/iframe/RankingDetails/238?disciplineId=10&groupId=1&momentId=171431&disciplineSeasonId=414&rankingTypeId=2&categoryId=22&raceTypeId=0',
'2024':'https://dataride.uci.org/iframe/RankingDetails/238?disciplineId=10&groupId=1&momentId=185139&disciplineSeasonId=432&rankingTypeId=2&categoryId=22&raceTypeId=0',
'2025':'https://dataride.uci.org/iframe/RankingDetails/238?disciplineId=10&groupId=1&momentId=189422&disciplineSeasonId=444&rankingTypeId=2&categoryId=22&raceTypeId=0'}

In [ ]:
#Sometimes the teams you want to look at are scattered in the rankings (eg. January) so pull out the ones you want. First scrape the latest for each year, then will filter
df = pd.DataFrame()
base_url = 'https://dataride.uci.org'
for k in url_dict:
    print(k, url_dict[k])
    browser = init_browser()
    url = url_dict[k]
    browser.get(url)
    sleep(2)
    # Scrape the page
    soup = BeautifulSoup(browser.page_source, 'lxml')
    divs = soup.find('div', class_='uci-table-wrapper')
    links = divs.find('table') #gets the internal links to each team on the rankings page
    tdf = pd.read_html(StringIO(str(links)), header=0, index_col = None)[0]
    tdf1 = pd.read_html(StringIO(str(links)), header=0, index_col = None, extract_links="body")[0]
    tdf1 = tdf1.map(lambda x: f"{base_url}{x[1]}" if isinstance(x, tuple) and x[1] else x[0] if isinstance(x, tuple) else x)
    tdf['URL'] = tdf1.Name
    print(tdf.head())
    tdf['Year'] = k
    df = pd.concat([df, tdf], axis=0)

2023 https://dataride.uci.org/iframe/RankingDetails/238?disciplineId=10&groupId=1&momentId=171431&disciplineSeasonId=414&rankingTypeId=2&categoryId=22&raceTypeId=0
  Rank                     Name Nation    Points  \
0    1  UAE TEAM EMIRATES (UAD)    UAE  30963.18   
1    1                      NaN    NaN       NaN   
2    2        JUMBO-VISMA (TJV)    NED  29654.45   
3    2                      NaN    NaN       NaN   
4    3  SOUDAL QUICK-STEP (SOQ)    BEL  18702.85   

                                                 URL  
0  https://dataride.uci.org/iframe/TeamRankingDet...  
1                                                     
2  https://dataride.uci.org/iframe/TeamRankingDet...  
3                                                     
4  https://dataride.uci.org/iframe/TeamRankingDet...  
2024 https://dataride.uci.org/iframe/RankingDetails/238?disciplineId=10&groupId=1&momentId=185139&disciplineSeasonId=432&rankingTypeId=2&categoryId=22&raceTypeId=0
  Rank                       

In [ ]:
df = df.dropna()
df

,Rank,Name,Nation,Points,URL,Year
0,1,UAE TEAM EMIRATES (UAD),UAE,30963.18,https://dataride.uci.org/iframe/TeamRankingDet...,2023
2,2,JUMBO-VISMA (TJV),NED,29654.45,https://dataride.uci.org/iframe/TeamRankingDet...,2023
4,3,SOUDAL QUICK-STEP (SOQ),BEL,18702.85,https://dataride.uci.org/iframe/TeamRankingDet...,2023
6,4,INEOS GRENADIERS (IGD),GBR,17760.93,https://dataride.uci.org/iframe/TeamRankingDet...,2023
8,5,LIDL-TREK (LTK),USA,16054.45,https://dataride.uci.org/iframe/TeamRankingDet...,2023
...,...,...,...,...,...,...
70,36 -2,MOVISTAR BEST PC (MBP),ECU,290.00,https://dataride.uci.org/iframe/TeamRankingDet...,2025
72,37 +6,XDS ASTANA DEVELOPMENT TEAM (AQD),KAZ,284.00,https://dataride.uci.org/iframe/TeamRankingDet...,2025
74,38 -3,ROOJAI INSURANCE (R0I),THA,279.66,https://dataride.uci.org/iframe/TeamRankingDet...,2025
76,39 -1,UAE TEAM EMIRATES GEN-Z (UAZ),UAE,271.00,https://dataride.uci.org/iframe/TeamRankingDet...,2025


In [ ]:
#This is a big, fat kludge because str.split wasn't working on the series
team_codes = []
for i in df['URL']:
    team_codes.append(str(i.split('TeamRankingDetails/')[1].split('?')[0]))
df['Team Code'] = team_codes
df.head()

,Rank,Name,Nation,Points,URL,Year,Team Code
0,1,UAE TEAM EMIRATES (UAD),UAE,30963.18,https://dataride.uci.org/iframe/TeamRankingDet...,2023,595
2,2,JUMBO-VISMA (TJV),NED,29654.45,https://dataride.uci.org/iframe/TeamRankingDet...,2023,538
4,3,SOUDAL QUICK-STEP (SOQ),BEL,18702.85,https://dataride.uci.org/iframe/TeamRankingDet...,2023,495
6,4,INEOS GRENADIERS (IGD),GBR,17760.93,https://dataride.uci.org/iframe/TeamRankingDet...,2023,1552
8,5,LIDL-TREK (LTK),USA,16054.45,https://dataride.uci.org/iframe/TeamRankingDet...,2023,1628


In [ ]:
# Filter the DF to the desired team codes

In [ ]:
top_teams = ['595', '1249', '467', '1628', '1554', '2125', '1552', '1084', '2132', '495', '1717', '3311', '987', '563', '1377', '3421', '562', '640', '538', '596', '1488', '1277', '1372', '929']

In [ ]:
top_df = df[df['Team Code'].isin(top_teams)]
top_df

,Rank,Name,Nation,Points,URL,Year,Team Code
0,1,UAE TEAM EMIRATES (UAD),UAE,30963.18,https://dataride.uci.org/iframe/TeamRankingDet...,2023,595
2,2,JUMBO-VISMA (TJV),NED,29654.45,https://dataride.uci.org/iframe/TeamRankingDet...,2023,538
4,3,SOUDAL QUICK-STEP (SOQ),BEL,18702.85,https://dataride.uci.org/iframe/TeamRankingDet...,2023,495
6,4,INEOS GRENADIERS (IGD),GBR,17760.93,https://dataride.uci.org/iframe/TeamRankingDet...,2023,1552
8,5,LIDL-TREK (LTK),USA,16054.45,https://dataride.uci.org/iframe/TeamRankingDet...,2023,1628
...,...,...,...,...,...,...,...
38,20,TEAM PICNIC POSTNL (TPP),NED,1118.00,https://dataride.uci.org/iframe/TeamRankingDet...,2025,596
40,21,ALPECIN-DECEUNINCK (ADC),BEL,1069.00,https://dataride.uci.org/iframe/TeamRankingDet...,2025,1488
42,22,ARKEA-B&B HOTELS (ARK),FRA,1020.00,https://dataride.uci.org/iframe/TeamRankingDet...,2025,1277
44,23,INTERMARCHÉ - WANTY (IWA),BEL,899.00,https://dataride.uci.org/iframe/TeamRankingDet...,2025,1372


In [ ]:
# prompt: create a pivot table from top_df with Name as row, Year as column, Points as value

pivot_table = pd.pivot_table(top_df, values='Points', index='Team Code', columns='Year')
pivot_table


Year,2023,2024,2025
Team Code,,,
1084,9096.00,15930.84,1764.01
1249,7044.44,6563.24,3583.66
1277,7229.00,8735.00,1020.00
1372,10492.28,9915.00,899.00
1377,3397.67,3603.81,1657.00
1488,14519.25,15020.00,1069.00
1552,17760.93,15547.97,3083.60
1554,13325.13,16894.33,2667.26
1628,16054.45,17989.00,2404.72


In [ ]:
# #Go to the UCI team rankings page and get a bunch of information about teams listed (limiting to top 22) including the name
# browser = init_browser()
# m = '189265' #Change to latest ranking moment from URL on dataride page
# y = '444' #CHange to desired year code (disciplineSeasonId) in URL
# #2023
# #2024_url='https://dataride.uci.org/iframe/RankingDetails/238?disciplineId=10&groupId=1&momentId=185139&disciplineSeasonId=432&rankingTypeId=2&categoryId=22&raceTypeId=0'
# url = f'https://dataride.uci.org/iframe/RankingDetails/238?disciplineId=10&groupId=1&momentId={m}&disciplineSeasonId={y}&rankingTypeId=2&categoryId=22&raceTypeId=0'
# team_ranking_urls = []
# browser.get(url)
# sleep(1)
# # Scrape the page
# soup = BeautifulSoup(browser.page_source, 'lxml')
# divs = soup.find('div', class_='uci-table-wrapper')
# links = divs.find('table').find_all('a', href=True) #gets the internal links to each team on the rankings page
# team_links = [base_url + a['href'] for a in links] #adds the base URL to formulate a hyperlink
# team_links_name = [a.string for a in links] # a list of the team names linked to match with the hyperlinks
# team_code = [a.split('/')[5].split('?')[0] for a in team_links] #IMPORTANT the API numeric ID for each team,
# #, follows team from year to year regardless of name changes
# teams = [i.split(' (')[0] for i in team_links_name] #A list of team names only without the TLA


In [ ]:
team_code_dict = top_df.set_index('Team Code')['Name'].to_dict()
team_code_dict


{'595': 'UAE TEAM EMIRATES XRG (UAD)',
 '538': 'TEAM VISMA | LEASE A BIKE (TVL)',
 '495': 'SOUDAL QUICK-STEP (SOQ)',
 '1552': 'INEOS GRENADIERS (IGD)',
 '1628': 'LIDL-TREK (LTK)',
 '2125': 'BAHRAIN VICTORIOUS (TBV)',
 '562': 'GROUPAMA-FDJ (GFC)',
 '1488': 'ALPECIN-DECEUNINCK (ADC)',
 '929': 'LOTTO (LOT)',
 '1554': 'RED BULL - BORA - HANSGROHE (RBH)',
 '987': 'EF EDUCATION - EASYPOST (EFE)',
 '467': 'MOVISTAR TEAM (MOV)',
 '1717': 'TEAM JAYCO ALULA (JAY)',
 '1372': 'INTERMARCHÉ - WANTY (IWA)',
 '563': 'COFIDIS (COF)',
 '3421': 'ISRAEL - PREMIER TECH (IPT)',
 '596': 'TEAM PICNIC POSTNL (TPP)',
 '1084': 'DECATHLON AG2R LA MONDIALE TEAM (DAT)',
 '1277': 'ARKEA-B&B HOTELS (ARK)',
 '1249': 'XDS ASTANA TEAM (XAT)',
 '2132': 'UNO-X MOBILITY (UXM)',
 '640': 'TOTALENERGIES (TEN)',
 '1377': 'Q36.5 PRO CYCLING TEAM (Q36)',
 '3311': 'TUDOR PRO CYCLING TEAM (TUD)'}

In [ ]:
pivot_table['Name'] = pivot_table.index.map(team_code_dict)
pivot_table['Current Total'] = pivot_table['2023'] + pivot_table['2024']
pivot_table.sort_values(by='Current Total', ascending=False, inplace=True)
pivot_table

Year,2023,2024,2025,Name,Current Total
Team Code,,,,,
595,30963.18,37407.60,5757.13,UAE TEAM EMIRATES XRG (UAD),68370.78
538,29654.45,20427.98,2022.99,TEAM VISMA | LEASE A BIKE (TVL),50082.43
495,18702.85,18153.97,1991.74,SOUDAL QUICK-STEP (SOQ),36856.82
1628,16054.45,17989.00,2404.72,LIDL-TREK (LTK),34043.45
1552,17760.93,15547.97,3083.60,INEOS GRENADIERS (IGD),33308.90
1554,13325.13,16894.33,2667.26,RED BULL - BORA - HANSGROHE (RBH),30219.46
1488,14519.25,15020.00,1069.00,ALPECIN-DECEUNINCK (ADC),29539.25
562,14834.51,12357.00,1419.00,GROUPAMA-FDJ (GFC),27191.51
929,14112.83,12579.29,654.00,LOTTO (LOT),26692.12


In [ ]:
# prompt: add a column to pivot_table with ranking of Name by Current Total

pivot_table['Rank'] = pivot_table['Current Total'].rank(ascending=False, method='min')
pivot_table


Year,2023,2024,2025,Name,Current Total,Rank
Team Code,,,,,,
595,30963.18,37407.60,5757.13,UAE TEAM EMIRATES XRG (UAD),68370.78,1.0
538,29654.45,20427.98,2022.99,TEAM VISMA | LEASE A BIKE (TVL),50082.43,2.0
495,18702.85,18153.97,1991.74,SOUDAL QUICK-STEP (SOQ),36856.82,3.0
1628,16054.45,17989.00,2404.72,LIDL-TREK (LTK),34043.45,4.0
1552,17760.93,15547.97,3083.60,INEOS GRENADIERS (IGD),33308.90,5.0
1554,13325.13,16894.33,2667.26,RED BULL - BORA - HANSGROHE (RBH),30219.46,6.0
1488,14519.25,15020.00,1069.00,ALPECIN-DECEUNINCK (ADC),29539.25,7.0
562,14834.51,12357.00,1419.00,GROUPAMA-FDJ (GFC),27191.51,8.0
929,14112.83,12579.29,654.00,LOTTO (LOT),26692.12,9.0


In [ ]:
# prompt: create a column in pivot_tables showing the Current Total difference to the row with Rank 18

# Calculate the difference from the row with Rank 18
rank_18_total = pivot_table[pivot_table['Rank'] == 18]['Current Total'].iloc[0]
pivot_table['Difference from Rank 18'] = pivot_table['Current Total'] - rank_18_total
pivot_table

Year,2023,2024,2025,Name,Current Total,Rank,Difference from Rank 18
Team Code,,,,,,,
595,30963.18,37407.60,5757.13,UAE TEAM EMIRATES XRG (UAD),68370.78,1.0,50043.53
538,29654.45,20427.98,2022.99,TEAM VISMA | LEASE A BIKE (TVL),50082.43,2.0,31755.18
495,18702.85,18153.97,1991.74,SOUDAL QUICK-STEP (SOQ),36856.82,3.0,18529.57
1628,16054.45,17989.00,2404.72,LIDL-TREK (LTK),34043.45,4.0,15716.20
1552,17760.93,15547.97,3083.60,INEOS GRENADIERS (IGD),33308.90,5.0,14981.65
1554,13325.13,16894.33,2667.26,RED BULL - BORA - HANSGROHE (RBH),30219.46,6.0,11892.21
1488,14519.25,15020.00,1069.00,ALPECIN-DECEUNINCK (ADC),29539.25,7.0,11212.00
562,14834.51,12357.00,1419.00,GROUPAMA-FDJ (GFC),27191.51,8.0,8864.26
929,14112.83,12579.29,654.00,LOTTO (LOT),26692.12,9.0,8364.87


In [ ]:
# prompt: for each Name in pivot_table generate a projection of Current Total for the next 12 weeks based on a weekly average equalling each Name's 2025 value divided by 8

# Calculate weekly average for 2025
pivot_table['Weekly Average'] = pivot_table['2025'] / 8 #BE SURE TO UPDATE WITH THE NUMBER OF WEEKS IN THE RANKINGS SO FAR

# Generate projection for the next 30 weeks

pivot_table['Projection'] = pivot_table['Weekly Average'] * 30 + pivot_table['Current Total']

# Sort the DataFrame by the projection in descending order
pivot_table.sort_values(by='Projection', ascending=False, inplace=True)
pivot_table['Projected Rank'] = pivot_table['Projection'].rank(ascending=False, method='min')
pivot_table['Change'] = pivot_table['Rank'] - pivot_table['Projected Rank']
pivot_table
# Display the updated pivot table
pivot_table


Year,2023,2024,2025,Name,Current Total,Rank,Difference from Rank 18,Weekly Average,Projection,Projected Rank,Change
Team Code,,,,,,,,,,,
595,30963.18,37407.60,5757.13,UAE TEAM EMIRATES XRG (UAD),68370.78,1.0,50043.53,719.64125,89960.0175,1.0,0.0
538,29654.45,20427.98,2022.99,TEAM VISMA | LEASE A BIKE (TVL),50082.43,2.0,31755.18,252.87375,57668.6425,2.0,0.0
1552,17760.93,15547.97,3083.60,INEOS GRENADIERS (IGD),33308.90,5.0,14981.65,385.45000,44872.4000,3.0,2.0
495,18702.85,18153.97,1991.74,SOUDAL QUICK-STEP (SOQ),36856.82,3.0,18529.57,248.96750,44325.8450,4.0,-1.0
1628,16054.45,17989.00,2404.72,LIDL-TREK (LTK),34043.45,4.0,15716.20,300.59000,43061.1500,5.0,-1.0
1554,13325.13,16894.33,2667.26,RED BULL - BORA - HANSGROHE (RBH),30219.46,6.0,11892.21,333.40750,40221.6850,6.0,0.0
2125,15787.81,9547.16,2302.00,BAHRAIN VICTORIOUS (TBV),25334.97,10.0,7007.72,287.75000,33967.4700,7.0,3.0
1488,14519.25,15020.00,1069.00,ALPECIN-DECEUNINCK (ADC),29539.25,7.0,11212.00,133.62500,33548.0000,8.0,-1.0
562,14834.51,12357.00,1419.00,GROUPAMA-FDJ (GFC),27191.51,8.0,8864.26,177.37500,32512.7600,9.0,-1.0


# **Gather results for deeper analysis**

In [ ]:
#build all the team_rider URLS - this is the only way that works. Scraping the team URLs from the rankings page only reflects the most recent ranking
#Women rankingId=35, groupId=2
disciplineSeasonIds = ['444'] # 2023-5 season IDs 444: '2025', 432: '2024', 414: '2023'
moments = ['189422'] #['185139'] #Get the moment ID from one of the team URLS on the ranking page date you want
date = ['2025'] #Year
ranking_urls = []
rdate = []
rteam = []
for index, value in enumerate(disciplineSeasonIds):
    s = value
    m = moments[index]
    d = date[index]
    for t in top_teams:
        team = team_code_dict[t] #alter to suit the year you're using
        rteam.append(team)
        rdate.append(d)
        ranking_url = f"https://dataride.uci.org/iframe/TeamRankingDetails/{t}?rankingId=238&groupId=1&momentId={m}&baseRankingTypeId=2&disciplineSeasonId={s}&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0"
        ranking_urls.append(ranking_url)

In [ ]:
len(ranking_urls)

24

In [ ]:
#Limted to top 22 teams, sho
ranking_urls


['https://dataride.uci.org/iframe/TeamRankingDetails/595?rankingId=238&groupId=1&momentId=189422&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0',
 'https://dataride.uci.org/iframe/TeamRankingDetails/1249?rankingId=238&groupId=1&momentId=189422&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0',
 'https://dataride.uci.org/iframe/TeamRankingDetails/467?rankingId=238&groupId=1&momentId=189422&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0',
 'https://dataride.uci.org/iframe/TeamRankingDetails/1628?rankingId=238&groupId=1&momentId=189422&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0',
 'https://dataride.uci.org/iframe/TeamRankingDetails/1554?rankingId=238&groupId=1&momentId=189422&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&rac

In [ ]:
# Re-run any problem teams that were missed
ranking_urls2 = ['https://dataride.uci.org/iframe/TeamRankingDetails/1488?rankingId=238&groupId=1&momentId=169177&baseRankingTypeId=2&disciplineSeasonId=414&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0','https://dataride.uci.org/iframe/TeamRankingDetails/1554?rankingId=238&groupId=1&momentId=169177&baseRankingTypeId=2&disciplineSeasonId=414&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0']

## **Go to each team URL list to get links to individual riders**

In [ ]:
# The lists of information are: result_urls (URL to week ranking riders with points) rdate (ranking date) rteam (duplicated in URL)
# NOTE the dataride system is laggy so put in some sleep time. Sometimes it a page will come up empty, so capture those URLs into a list of ones to re-run
problem_list = []
rider_url_list = []
base_url = 'https://dataride.uci.org'
for index, value in enumerate(ranking_urls):
    print(value)
    team=team_code_dict[value.split('?')[0].split('TeamRankingDetails/')[1].split('\'')[0]] #change to suit the team code dictionary for the year you want
    print(f"Getting result links for {team}")
    driver = init_browser()
    driver.get(value)

    # # Scrape the page
    try:
        sleep(5)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        divs = soup.find('div', class_='uci-table-wrapper')
        tbl = divs.find('table')
        links = tbl.find_all('a', href=True)
        rp_df = pd.read_html(StringIO(str(tbl)), header=0, index_col = None)[0]
        print(rp_df.head(1))
    # Assemble the team-ranking-rider-results URLs into a dataframe
        rp_df['URL'] = [base_url + a['href'] for a in links]
        rp_df['Date'] = rdate[index]
        if len(rp_df)>0:
            rider_url_list.append(rp_df)
        else:
            problem_list.append(value)

    except:
        print(f"Problem with {value}")

        pass


https://dataride.uci.org/iframe/TeamRankingDetails/595?rankingId=238&groupId=1&momentId=189422&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0
Getting result links for UAE TEAM EMIRATES XRG (UAD)
                       Rider Nation  Age  Points
0  AYUSO PESQUERA Juan (UAD)    ESP   23  1045.0
https://dataride.uci.org/iframe/TeamRankingDetails/1249?rankingId=238&groupId=1&momentId=189422&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0
Getting result links for XDS ASTANA TEAM (XAT)
                     Rider Nation  Age  Points
0  SCARONI Christian (XAT)    ITA   28   795.0
https://dataride.uci.org/iframe/TeamRankingDetails/467?rankingId=238&groupId=1&momentId=189422&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0
Getting result links for MOVISTAR TEAM (MOV)
                      Rider Nation  Age  Points
0  ROMO 

KeyboardInterrupt: 

In [ ]:
#Look at the first dataframe in the list
len(rider_url_list)

0

In [ ]:
# Put all the rider links together into one big dataframe so we can then use the URLs to get results
df = pd.DataFrame()
for d in rider_url_list:
    df = pd.concat([d, df], axis=0)


In [ ]:
len(df)

0

In [ ]:
#Split the team out into a new column
df[['Rider','Team']] = df['Rider'].str.split(pat='\(', n=1, expand=True)
df['Team'] = df['Team'].str.replace(')','')

In [ ]:
df.groupby(['Date','Team'])['Rider'].count().reset_index()

,Date,Team,Rider
0,2025,ADC,13
1,2025,ARK,15
2,2025,COF,18
3,2025,DAT,20
4,2025,EFE,17
5,2025,GFC,20
6,2025,IGD,20
7,2025,IPT,20
8,2025,IWA,19
9,2025,JAY,20


In [ ]:
problem_list

['https://dataride.uci.org/iframe/TeamRankingDetails/2125?rankingId=238&groupId=1&momentId=189265&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0']

In [ ]:
# Save out the URLs
outfile = '2020-2022_rider_urls.csv'
with open(outfile, 'w') as file:

    try:

        file.write(df.to_csv(header=True, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Fix any missing URLs

Manually adding missing team URL

In [ ]:
# problem_list = ['https://dataride.uci.org/iframe/TeamRankingDetails/495?rankingId=238&groupId=1&momentId=184109&baseRankingTypeId=2&disciplineSeasonId=432&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0',
#  'https://dataride.uci.org/iframe/TeamRankingDetails/1554?rankingId=238&groupId=1&momentId=184109&baseRankingTypeId=2&disciplineSeasonId=432&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0',
#  'https://dataride.uci.org/iframe/TeamRankingDetails/1552?rankingId=238&groupId=1&momentId=184109&baseRankingTypeId=2&disciplineSeasonId=432&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0']
rider_url_list = []
base_url = 'https://dataride.uci.org'
for index, value in enumerate(problem_list):
    team=team_code_dict[value.split('?')[0].split('TeamRankingDetails/')[1].split('\'')[0]]
    print(f"Getting result links for {team}")
    driver = init_browser()
    driver.get(value)
    sleep(3)
    # # Scrape the page
    try:
        soup = BeautifulSoup(driver.page_source, 'lxml')
        divs = soup.find('div', class_='uci-table-wrapper')
        tbl = divs.find('table')
        links = tbl.find_all('a', href=True)
        rp_df = pd.read_html(str(tbl), header=0, index_col = None)[0]
        print(rp_df.head(20))
# Assemble the team-ranking-rider-results URLs into a dataframe
        rp_df['URL'] = [base_url + a['href'] for a in links]
        # rp_df['Date'] = rdate[index]
        rider_url_list.append(rp_df)

    except:
        print(f"Problem with {value}")
        problem_list.append(value)
        pass

In [ ]:
rider_urls = df.URL.values.tolist()
rider_urls

In [ ]:
rider_url_list[]

In [ ]:
df2 = pd.DataFrame()
for d in rider_url_list:
    df2 = pd.concat([d, df2], axis=0)

In [ ]:
len(df2)

In [ ]:
# df2[['Rider','Team']] = df2['Rider'].str.split('\(', 1, expand=True)
# df2.Team = df2['Team'].str.replace('\)','')
#The above code doesn't work anymore see above

In [ ]:
df2.head()

Or, upload the URL list


In [ ]:
for item in uploaded:
    url_df = pd.read_csv(item, header=0)

In [ ]:
url_df = pd.concat([url_df, df], axis=0)

In [ ]:
url_df.head()
# url_df.drop(columns=['Team'], inplace=True)

In [ ]:
# url_df[['Rider','Team']] = url_df['Rider'].str.split('\(', 1, expand=True)

In [ ]:
url_df[['Rider','Team']] = url_df['Rider'].str.split('\(', 1, expand=True)

In [ ]:
url_df['Team'].value_counts()

# **Get rider results per ranking**



In [ ]:
# UPLOAD the URL list if saved
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))

Saving 2020-2022_rider_urls.csv to 2020-2022_rider_urls.csv
User uploaded file "2020-2022_rider_urls.csv"


In [ ]:
for item in uploaded:
    df = pd.read_csv(item, header=0)
#OR use what is in this session: Get the URLs into a list to go through




NameError: name 'uploaded' is not defined

In [ ]:
rider_urls2 = list(set(rider_urls2))
len(rider_urls2)

In [ ]:
rider_urls = df.URL.tolist()
len(rider_urls)


409

In [ ]:
# rider_urls[0].split('disciplineSeasonId=')[1].split('&')[0]

'145'

In [ ]:
# prompt: find key for value 145 in year_dict
# year_dict.get('2020')


In [ ]:
# len(rider_urls)

220

## **Get rider results**

In [ ]:
#TODO: Make this into a function ffs
# def get_results(rider_urls):

# year_dict = {'159':'2022','150':'2021','145':'2020'}
rider_df_list = []
problem_list
problem_list2 = ['https://dataride.uci.org/iframe/RiderRankingDetails/14575?rankingId=1&groupId=1&momentId=52756&baseRankingTypeId=2&disciplineSeasonId=145&disciplineId=10&categoryId=22&raceTypeId=0&countryId=63&teamId=467',
 'https://dataride.uci.org/iframe/RiderRankingDetails/27114?rankingId=1&groupId=1&momentId=140531&baseRankingTypeId=2&disciplineSeasonId=150&disciplineId=10&categoryId=22&raceTypeId=0&countryId=31&teamId=2125',
 'https://dataride.uci.org/iframe/RiderRankingDetails/100965?rankingId=1&groupId=1&momentId=140531&baseRankingTypeId=2&disciplineSeasonId=150&disciplineId=10&categoryId=22&raceTypeId=0&countryId=31&teamId=2125',
 'https://dataride.uci.org/iframe/RiderRankingDetails/111440?rankingId=1&groupId=1&momentId=140531&baseRankingTypeId=2&disciplineSeasonId=150&disciplineId=10&categoryId=22&raceTypeId=0&countryId=187&teamId=595']
penalties = []
rider_penalty = []
for url in rider_urls: #problem_list2
    season_id = url.split('disciplineSeasonId=')[1].split('&')[0]
    year = '2025'
    print(f"Getting results for {url} from {year}")
    driver = init_browser()
    driver.get(url)
    sleep(3)

    # Scrape the page
    try:
        soup = BeautifulSoup(driver.page_source, 'lxml')
        rider = soup.find('p', string='Age').parent.find_previous_sibling('p').string
        try:
            penalty = soup.find('p', string='Sanctions').find_next('span').string
            penalties.append(penalty)
            rider_penalty.append(rider)
            print(f"Rider {rider} was penalized {penalty} points")
        except:
            penalties.append('0')
            rider_penalty.append(rider)
            pass
        divs = soup.find('div', class_='uci-table-wrapper')
        tbl = divs.find('table')
        rider_results = pd.read_html(StringIO(str(tbl)), header=0, index_col = None)[0]
        if len(rider_results) == 0:
            print(f"Promlem getting results for {rider}")
            problem_list.append(url)
        else:
            print(f"Got results for {rider}")
            rider_results = rider_results.dropna(how='all')
            rider_results['Name'] = rider
            rider_results['Year'] = year
            rider_df_list.append(rider_results)

    except:
        print(f"Problem with {url}")
        problem_list.append(url)
        pass


    # return pdf, df_results, problem_list

Getting results for https://dataride.uci.org/iframe/RiderRankingDetails/152803?rankingId=1&groupId=1&momentId=189265&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=21&teamId=929 from 2025
Got results for VAN MOER Brent (LOT)
Getting results for https://dataride.uci.org/iframe/RiderRankingDetails/1720439?rankingId=1&groupId=1&momentId=189265&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=21&teamId=929 from 2025
Got results for VAN EETVELT Lennert (LOT)
Getting results for https://dataride.uci.org/iframe/RiderRankingDetails/735965?rankingId=1&groupId=1&momentId=189265&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=21&teamId=929 from 2025
Got results for DE LIE Arnaud (LOT)
Getting results for https://dataride.uci.org/iframe/RiderRankingDetails/1547884?rankingId=1&groupId=1&momentId=189265&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId

In [ ]:
pdf = pd.DataFrame({'Rider':rider_penalty,'Penalty':penalties})

df_results = pd.DataFrame()
for d in rider_df_list:
    df_results = pd.concat([d, df_results], axis=0)

    # return pdf, df_results, problem_list


In [ ]:
df_results[['Name','Team']] = df_results['Name'].str.split(pat='(', n=1, expand=True)
df_results['Team'] = df_results['Team'].str.replace(')','')

In [ ]:
df_results.groupby(['Year','Team'])['Name'].nunique().reset_index()

,Year,Team,Name
0,2025,ADC,13
1,2025,ARK,15
2,2025,COF,18
3,2025,DAT,20
4,2025,EFE,16
5,2025,GFC,20
6,2025,IGD,20
7,2025,IPT,20
8,2025,IWA,19
9,2025,JAY,20


In [ ]:
problem_list

['https://dataride.uci.org/iframe/TeamRankingDetails/2125?rankingId=238&groupId=1&momentId=189265&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=0&teamId=0',
 'https://dataride.uci.org/iframe/RiderRankingDetails/1547884?rankingId=1&groupId=1&momentId=189265&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=21&teamId=929',
 'https://dataride.uci.org/iframe/RiderRankingDetails/88143?rankingId=1&groupId=1&momentId=189265&baseRankingTypeId=2&disciplineSeasonId=444&disciplineId=10&categoryId=22&raceTypeId=0&countryId=193&teamId=987']

In [ ]:
outfile = '2025_penalties.csv'
with open(outfile, 'w') as file:

    try:

        file.write(pdf.to_csv(header=False, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)
outfile2 = '2025_results.csv'
with open(outfile2, 'w') as file:

    try:

        file.write(df_results.to_csv(header=True, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#TODO: Make this into a function ffs
year_dict = {'159':'2022','150':'2021','145':'2020'}
problem_list = []
penalties = []

# problem_list = []
# url = 'https://dataride.uci.org//iframe/RiderRankingDetails/87061?rankingId=1&groupId=1&momentId=152874&baseRankingTypeId=2&disciplineSeasonId=159&disciplineId=10&categoryId=22&raceTypeId=0&countryId=21&teamId=495'
for url in rider_urls:

    # rider = df.iloc[index,0]
# yr = df.iloc[index,5]
    # print(f"Getting results for {rider} in {yr}")
    driver = init_browser()
    driver.get(url)
    sleep(2)

    # Scrape the page
    try:
        soup = BeautifulSoup(driver.page_source, 'lxml')
        rider = soup.find('p', string='Age').parent.find_previous_sibling('p').string
        try:
            penalty = soup.find('p', string='Sanctions').find_next('span').string
            penalties.append(penalty)
            rider_penalty.append(rider)
            print(f"Rider {rider} was penalized {penalty} points")
        except:
            penalties.append('0')
            rider_penalty.append(rider)
            pass
        divs = soup.find('div', class_='uci-table-wrapper')
        tbl = divs.find('table')
        rider_results = pd.read_html(StringIO(str(tbl)), header=0, index_col = None)[0]
        if len(rider_results) == 0:
            print(f"Promlem getting results for {rider}")
            problem_list.append(url)
        else:
            print(f"Got results for {rider}")
            rider_results = rider_results.dropna(how='all')
            rider_results['Name'] = rider
            rider_results['Year'] = year
            rider_df_list.append(rider_results)

    except:
        print(f"Problem with {url}")
        problem_list.append(url)
        pass
pdf = pd.DataFrame({'Rider':rider_penalty,'Penalty':penalties})
outfile = '2024_penalties.csv'
with open(outfile, 'w') as file:

    try:

        file.write(pdf.to_csv(header=False, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)
df_results = pd.DataFrame()
for d in rider_df_list:
    df_results = pd.concat([d, df_results], axis=0)
outfile2 = '2024_results.csv'
with open(outfile2, 'w') as file:

    try:

        file.write(df_results.to_csv(header=True, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile2)

In [ ]:
pdf = pd.DataFrame({'Rider':rider_penalty,'Penalty':penalties})
outfile = '2024_penalties.csv'
with open(outfile, 'w') as file:

    try:

        file.write(pdf.to_csv(header=False, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)

In [ ]:
pdf[['Rider','Team']] = pdf['Rider'].str.split(pat='(', n=1, expand=True)
pdf['Team'] = pdf['Team'].str.replace(')','')

In [ ]:
df_results[['Name','Team']] = df_results['Name'].str.split(pat='(', n=1, expand=True)
df_results['Team'] = df_results['Team'].str.replace(')','')

In [ ]:
# prompt: change pdf.Penalty to number

pdf['Penalty'] = pd.to_numeric(pdf['Penalty'], errors='coerce').fillna(0).astype(int)

In [ ]:
pdf.groupby('Team')['Penalty'].sum()

In [ ]:
outfile = '2024_results_final.csv'
with open(outfile, 'w') as file:

    try:

        file.write(df_results.to_csv(header=True, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)

In [ ]:
# prompt: Find Riders in df that are not in df_results.Name

# Assuming df and df_results are already defined DataFrames
# and they both have a 'Name' column.

riders_not_in_results = df[~df['Rider'].isin(df_results['Name'])]
problem_list = riders_not_in_results.URL.to_list()

In [ ]:
df_results.groupby('Team')['Name'].nunique()

In [ ]:
# prompt: change df_results.Points into number

df_results['Points'] = pd.to_numeric(df_results['Points'], errors='coerce')

In [ ]:
df_results[df_results.Team=='JAY'].groupby('Name')['Points'].sum().sort_values(ascending=False)

# **Find any problem URLs fix and re-run**

In [ ]:
problem_list

In [ ]:
get_results(problem_list)

NameError: name 'get_results' is not defined

In [ ]:
probs=pd.DataFrame()
probs['URL'] = problem_list
probs.head()

# **Total up the penalties for each team**

In [ ]:
penalty_df = pd.DataFrame({'Rider':rider_penalty, 'Penalty':penalties})
penalty_df['Penalty'] = penalty_df['Penalty'].astype(float)

In [ ]:
penalty_df = penalty_df[penalty_df.Penalty > 0]
len(penalty_df)

8

In [ ]:
penalty_df[['Rider','Team']] = penalty_df['Rider'].str.split(pat='(', n=1, expand=True)
penalty_df['Team'] = penalty_df['Team'].str.replace(')','')

In [ ]:
penalty_df.groupby('Team')['Penalty'].sum()

,Penalty
Team,
ADC,15.0
ARK,15.0
COF,15.0
EFE,15.0
MOV,10.0
RBH,10.0
TEN,5.0
TUD,50.0


In [ ]:
df_results[['Team','Name']].drop_duplicates().shape[0]

407

In [ ]:
outfile = '2024_results_men.csv'
with open(outfile, 'w') as file:

    try:

        file.write(df_results.to_csv(header=True, index=False, encoding='UTF-8'))
    except:
        pass
    files.download(outfile)

In [ ]:
len(out_df2)

# Save all the results

In [ ]:
outfile = '2023_rider_results_10_23_corrected.csv'
with open(outfile, 'w') as file:

    try:

        file.write(out_df.to_csv(header=True, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)

# FIX TEAMS MISSING

In [ ]:
#TODO: Make this into a function ffs

rider_df_list = []
year = 2023

penalties = []
rider_penalty = []
# problem_list = []
# url = 'https://dataride.uci.org//iframe/RiderRankingDetails/87061?rankingId=1&groupId=1&momentId=152874&baseRankingTypeId=2&disciplineSeasonId=159&disciplineId=10&categoryId=22&raceTypeId=0&countryId=21&teamId=495'
for url in ipt_list:

# rider = df.iloc[index,0]
# yr = df.iloc[index,5]
# print(f"Getting results for {rider} in {yr}")
    driver = init_browser()
    driver.get(url)
    sleep(2)

    # Scrape the page
    try:
        soup = BeautifulSoup(driver.page_source, 'lxml')
        rider = soup.find('p', string='Age').parent.find_previous_sibling('p').string
        try:
            penalty = soup.find('p', string='Sanctions').find_next('span').string
            penalties.append(penalty)
            rider_penalty.append(rider)
            print(f"Rider {rider} was penalized {penalty} points")
        except:
            penalties.append('0')
            rider_penalty.append(rider)
            pass
        divs = soup.find('div', class_='uci-table-wrapper')
        tbl = divs.find('table')
        rider_results = pd.read_html(str(tbl), header=0, index_col = None)[0]
        if len(rider_results) == 0:
            print("ZERO!")
            problem_list.append(url)
        else:
            print("Got results")
            rider_results = rider_results.dropna(how='all')
            rider_results['Name'] = rider
            rider_results['Year'] = year
            rider_df_list.append(rider_results)

    except:
        print(f"Problem with {url}")
        problem_list.append(url)
        pass
pdf = pd.DataFrame({'Rider':rider_penalty,'Penalty':penalties})
outfile = '2023_penalties.csv'
with open(outfile, 'w') as file:

    try:

        file.write(pdf.to_csv(header=False, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)
outfile = '2023_all_results.csv'
with open(outfile, 'w') as file:

    try:

        file.write(rider_df_list.to_csv(header=False, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)

In [ ]:
ipt_df = pd.DataFrame()
for d in rider_df_list:
    ipt_df = pd.concat([ipt_df, d], axis=0)

In [ ]:
len(astana_df.Name.unique())

In [ ]:
outfile = 'ipt_results.csv'
with open(outfile, 'w') as file:

    try:

        file.write(ipt_df.to_csv(header=False, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)

# Analyze the **results**

In [ ]:
#UPLOAD THE Results saved
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))


In [ ]:
for item in uploaded:
    df1 = pd.read_csv(item, header=None)

In [ ]:
# df = df.rename(columns=({1:'Date',2:'Event',3:'Rank',4:'Points',5:'rider_team',6:'Year'}))

In [ ]:
#UPLOAD THE UCI Calendar for the year
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))

In [ ]:
for item in uploaded:
    df1 = pd.read_csv(item, header=0)

In [ ]:
df1.head()

In [ ]:
# rider_results[['Rider','Team']] = df['Rider'].str.split('\(', 1, expand=True)
# rider_results.Team = df['Team'].str.replace('\)','')

TO DO

1.   Map UCI calendar to downloaded results
2.   Map result codes to my codes



In [ ]:
outfile = 'all_rider_results.csv'
with open(outfile, 'w') as file:

    try:

        file.write(out_df.to_csv(header=False, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)

In [ ]:
problem_list

**WorldRanking**

In [ ]:
#Make wide rankings long
import pandas as pd
from google.colab import files

In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))


In [ ]:
for item in uploaded:
    df = pd.read_csv(item, header=0)

In [ ]:
df.head()

In [ ]:
vv = df.columns
df_long = pd.melt(df, id_vars='Position', value_vars=vv)

In [ ]:
df_long = pd.melt(df, id_vars='Position', value_vars=vv)

In [ ]:
df_long.head()

In [ ]:
outfile = 'wt_ps_stage_rankings_long.csv'
with open(outfile, 'w') as file:

    try:

        file.write(df_long.to_csv(header=False, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)

**analysing all rider results 2020-2022**

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
#UPLOAD THE RESULTS FILE
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))

In [ ]:
for item in uploaded:
    df = pd.read_csv(item, header=0)

In [ ]:
# df = df.rename(columns={0:'Date', 1:'Rank', 7:'Points', 9:'Rider',10:'Team',11:'Year',12:'Race',13:'Category'})
df.head()

In [ ]:
# df = df.drop(columns=[2,3,4,5,6,8], axis=0)

In [ ]:
# df.fillna(0, inplace=True)

In [ ]:
# df.loc[91450]

In [ ]:
# nongc = df[df.Date==0]
# nongc[nongc.Race==0]

In [ ]:
# gcpts = df[df.Points.str.contains("\+", na=False)]
# gcpts

In [ ]:
df[df.Team=='ACT'].groupby(['Team','Name'])['Points'].sum().sort_values(ascending=False)

In [ ]:
df.head()

In [ ]:
len(pt[pt.Year==2022])

In [ ]:
# e21 = df[(df.Year==2022) & (df.Team=='Israel-Premier Tech')]
# bv22 = e21.groupby('Rider')['Points'].sum()
# bv22.sort_values(ascending=False).reset_index()

In [ ]:
# len(df.Team.unique())

In [ ]:
#RENAME RESULTS FOR CHECKING POINTS
rdf = df

In [ ]:
#UPLOAD THE RANKING CODES
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))
for item in uploaded:
    rnk = pd.read_csv(item, header=0)

In [ ]:
for item in uploaded:
    rnk = pd.read_csv(item, header=0)

In [ ]:
rnk.head()

In [ ]:
rnk_nan = rnk.dropna()

In [ ]:
rnk_nan[rnk_nan.Code=='P SGC']

In [ ]:
rnk_nan[rnk_nan.Code=='CN ME RR']

In [ ]:
rdf.head()

In [ ]:
chk = rdf.merge(rnk_nan, on=['Rank','Code'], how="left")
chk

In [ ]:
chk.head()

In [ ]:
chk_missing = chk[chk.Points_x != chk.Points_y]
chk_missing

In [ ]:
len(chk_missing)

In [ ]:
race_list = chk_missing.groupby(['Race','Year'])['Code'].first()

In [ ]:
race_list.head(10)

In [ ]:
bad_races = race_list.reset_index().Race.unique()
bad_races

In [ ]:
for i in bad_races:
    print((chk_missing[chk_missing.Race==i]).head())

**Modeling different point systems**

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
#UPLOAD THE POINTS MODEL LONG DATA FORM

uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))

In [ ]:
for item in uploaded:
    ptm = pd.read_csv(item, header=0)

In [ ]:
# It probably came from wide data so get rid of the nulls
vv = ptm.columns
vv

In [ ]:
ptm = pd.melt(ptm, id_vars='Position', value_vars=vv)

In [ ]:
ptm = ptm.dropna()

In [ ]:
dfp = pd.DataFrame()

In [ ]:
# IF GOING WIDE TO LONG
dfp['Rank'] = ptm.Position
dfp['Code'] = ptm.variable
dfp['Points'] = ptm.value
#IF UPLOADING LONG
# dfp=ptm

In [ ]:
dfp.head()

In [ ]:
#UPLOAD RESULTS FILE
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))

In [ ]:
for item in uploaded:
    res = pd.read_csv(item, header=0)

In [ ]:
len(res)

In [ ]:
pmod = res.merge(dfp, on=['Rank','Code'], how="left")


In [ ]:
len(pmod)

In [ ]:
pmod.head()

In [ ]:
res.Type.unique()

In [ ]:
import numpy as np

In [ ]:
pt_mod = {'Stage Race':1, 'One Day':0.5, 'Championships':1, 'Grand Tour':1}
pmod['Pt Adj'] = np.nan
pmod['Pt Adj'] = pmod['Type'].map(pt_mod)
pmod.head()


In [ ]:
pmod['One Day adj'] = pmod.Points_y * pmod['Pt Adj']
pmod.head()

In [ ]:
rda = pmod.groupby(['Team','Type'])['One Day adj'].sum().reset_index()
# rda = pmod.groupby(['Team','Type'])['Points_y'].sum().reset_index()
rda

In [ ]:
rda.pivot(index='Team', columns='Type', values='One Day adj')
# rda.pivot(index='Team', columns='Type', values='Points_y')

In [ ]:
outfile = 'points_by_type_rankbin.csv'
with open(outfile, 'w') as file:

    try:

        file.write(pt.to_csv(header=False, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)

In [ ]:
#Analysis of results for fun


In [ ]:
rbc = pmod.groupby(['Team','Type'])['Points_y'].sum().reset_index()
rbc

In [ ]:
rbc.pivot(index='Team', columns='Type', values='Points_y')

In [ ]:
outfile = 'results by type.csv'
with open(outfile, 'w') as file:

    try:

        file.write(rbc.to_csv(header=False, index=False, encoding='UTF-8'))
    except:
        pass
files.download(outfile)